In [10]:
import pandas as pd
from faker import Faker
import random

# Inicializando o Faker e definindo a localização para Brasil
fake = Faker('pt_BR')

# Lista de cargos
cargos = [
    "Não",
    "Celebrações",
    "Técnica",
    "MIDAF - Dança",
    "Infantil",
    "Eventos/Cerimonial",
    "MILAF - Louvor",
    "Comunicação",
    "Intercessão",
    "Introdutores",
    "MJF - Jovens"
]

# Lista de sexos
sexos = ["Masculino", "Feminino"]

# Lista de estados civis
estados_civis = ["Solteiro(a)", "Casado(a)", "Divorciado(a)", "Viúvo(a)"]

# Lista de escolaridades
escolaridades = ["Fundamental", "Médio", "Superior", "Pós-graduação", "Mestrado", "Doutorado"]

# Função para gerar um telefone no padrão '(92)999961674'
def gerar_telefone():
    return f'(92)9{random.randint(1000, 9999)}{random.randint(1000, 9999)}'

# Função para gerar CEP de Manaus
def gerar_cep():
    return f'690{random.randint(10, 99)}-{random.randint(100, 999)}'

# Função para gerar endereço em Manaus
def gerar_endereco():
    return fake.street_name(), fake.bairro(), "Manaus", "AM", fake.building_number()

# Função para gerar "sim" ou "não" para receber notícias
def receber_noticias():
    return random.choice(["Sim", "Não"])

# Gerando dados
data = []
num_registros = 100  # Ajuste o número de registros conforme necessário

for i in range(num_registros):
    rua, bairro, cidade, uf, numero = gerar_endereco()
    registro = {
        'cpf':                      fake.cpf(),
        'nome_completo':            fake.name(),
        'data_nascimento':          fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
        'genero':                   random.choice(sexos),
        'numero_celular':           gerar_telefone(),
        'email':                    fake.email(),
        'estado_civil':             random.choice(estados_civis),
        'numero_celula':            random.randint(1, 10),
        'participacao_ministerio':  random.choice(cargos),
        'data_batismo':             fake.date_this_decade().strftime("%Y-%m-%d"),
        'data_conversao':           fake.date_this_year().strftime("%Y-%m-%d"),
        'escolaridade':             random.choice(escolaridades),
        'profissao':                fake.job(),
        'cep':                      gerar_cep(),
        'rua':                      rua,
        'bairro':                   bairro,
        'cidade':                   cidade,
        'uf':                       uf,
        'numero':                   random.randint(1, 999),
        'receber_noticias':         receber_noticias(),
        'id':                       i + 1
    }
    data.append(registro)

# Criando DataFrame
df = pd.DataFrame(data)

# Salvando em um arquivo CSV
df.to_csv('dados_falsos.csv', index=False)

print("Dados falsos gerados e salvos em 'dados_falsos.csv'.")


Dados falsos gerados e salvos em 'dados_falsos.csv'.


In [11]:
dados_falsos = pd.read_csv("dados_falsos.csv")
dados_falsos.head()

,cpf,nome_completo,data_nascimento,genero,numero_celular,email,estado_civil,numero_celula,participacao_ministerio,data_batismo,...,escolaridade,profissao,cep,rua,bairro,cidade,uf,numero,receber_noticias,id
0,934.216.507-99,Giovanna Câmara,1995-02-06,Masculino,(92)966448504,diogocardoso@example.net,Solteiro(a),5,Eventos/Cerimonial,2022-05-13,...,Médio,Mensageiro,69030-595,Aeroporto Casa Grande,Pindorama,Manaus,AM,327,Não,1
1,710.835.924-32,Cecilia Souza,1987-09-13,Masculino,(92)959772157,qoliveira@example.net,Viúvo(a),4,Técnica,2024-03-19,...,Fundamental,Webdesigner,69086-289,Morro Costa,Cidade Jardim Taquaril,Manaus,AM,995,Sim,2
2,805.367.241-80,Léo Barbosa,1950-07-05,Feminino,(92)990469258,gabriellyoliveira@example.net,Divorciado(a),3,MILAF - Louvor,2023-01-17,...,Médio,Paramédico,69054-216,Passarela Benjamin Pereira,Maria Tereza,Manaus,AM,201,Sim,3
3,751.624.938-64,Olivia Vargas,1986-03-24,Feminino,(92)939384464,piresarthur-miguel@example.net,Casado(a),4,MIDAF - Dança,2020-12-29,...,Mestrado,Escriturário,69071-918,Lagoa Stephany das Neves,Bandeirantes,Manaus,AM,522,Sim,4
4,824.906.153-51,José Rios,1987-06-30,Feminino,(92)936484816,joanasousa@example.net,Viúvo(a),10,Introdutores,2024-05-18,...,Médio,Marinheiro,69066-629,Loteamento de Ribeiro,Conjunto Serra Verde,Manaus,AM,129,Sim,5


In [14]:
import conexao

# Carregar os dados do CSV, ignorando a primeira linha de índices
df = pd.read_csv('dados_falsos.csv')

# Criar a conexão com o banco de dados
connection = conexao.create_connection()

if connection:
    cursor = connection.cursor()
    
    # Dropar todos os dados da tabela membros
    cursor.execute("DELETE FROM membros")
    connection.commit()
    print("Todos os dados da tabela membros foram deletados.")
    
    # Preparar a query de inserção
    insert_query = """
    INSERT INTO membros (
        cpf, 
        nome_completo, 
        data_nascimento, 
        genero, 
        numero_celular, 
        email,
        estado_civil, 
        numero_celula, 
        participacao_ministerio, 
        data_batismo,
        data_conversao, 
        escolaridade, 
        profissao, 
        cep, 
        rua, 
        bairro,
        cidade, 
        uf,
        numero,
        receber_noticias
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
    """
    
    # Iterar sobre as linhas do DataFrame e inserir os dados na tabela
    for index, row in df.iterrows():
        cursor.execute(insert_query, (
            row['cpf'],
            row['nome_completo'],
            row['data_nascimento'],
            row['genero'],
            row['numero_celular'],
            row['email'],
            row['estado_civil'],
            row['numero_celula'],
            row['participacao_ministerio'],
            row['data_batismo'],
            row['data_conversao'],
            row['escolaridade'],
            row['profissao'],
            row['cep'],
            row['rua'],
            row['bairro'],
            row['cidade'],
            row['uf'],
            row['numero'],
            row['receber_noticias']
        ))
    
    # Confirmar as alterações
    connection.commit()
    print("Dados inseridos na tabela membros com sucesso.")
    
    # Fechar a conexão
    conexao.close_connection(connection)
else:
    print("Falha ao conectar ao banco de dados.")


Conexão com o banco de dados estabelecida com sucesso!
Todos os dados da tabela membros foram deletados.
Dados inseridos na tabela membros com sucesso.
Conexão com o banco de dados foi encerrada.


In [16]:
df = dados_falsos

In [18]:
# Contagem de membros
total_membros = df['cpf'].nunique()
#numero_de_pessoas


In [24]:
# Contagem de profissões
todas_profissoes = df['profissao']
todas_profissoes = todas_profissoes.tolist()
#todas_profissoes

In [28]:
import folium
import requests

# Seus dados de exemplo com CEPs reais
data = {
    'cpf': ['934.216.507-99', '710.835.924-32', '805.367.241-80'],
    'nome_completo': ['Giovanna Câmara', 'Cecilia Souza', 'Léo Barbosa'],
    'cep': ['69030-595', '69086-289', '69054-216']
}

# Criar um DataFrame a partir dos seus dados
df = pd.DataFrame(data)

# Função para obter coordenadas a partir do CEP usando serviço de API pública
def get_coordinates(cep):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('localidade'), data.get('uf'), data.get('logradouro')
    else:
        return None, None, None

# Adicionar colunas de cidade, estado e logradouro com base no CEP
df['cidade'], df['uf'], df['logradouro'] = zip(*df['cep'].apply(get_coordinates))

# Filtrar registros onde não foi possível obter a localização
df = df.dropna(subset=['cidade', 'uf'])

# Criar o mapa com Folium
mapa = folium.Map(location=[-3.7934, -38.519], zoom_start=12)  # Coordenadas de exemplo, pode ajustar conforme necessário

# Adicionar marcadores para cada membro da igreja
for index, row in df.iterrows():
    address = f"{row['logradouro']}, {row['cidade']}, {row['uf']}, Brasil"
    location = geolocator.geocode(address)
    if location:
        folium.Marker([location.latitude, location.longitude], popup=row['nome_completo']).add_to(mapa)

# Salvar o mapa como um arquivo HTML
mapa.save('mapa_membros_igreja.html')

# Exibir o mapa (opcional)
mapa
